In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
%pip install openai
from openai import OpenAI   
# client = OpenAI(api_key=my_api_key)
client = OpenAI()  # API key will be read from environment variable OPENAI_API_KEY

In [3]:
def ask_question(prompt: str) -> str:
    """
    Ask a question to GPT, with moderation check.
    If the prompt is flagged, it will not be sent to the chat model.
    """
    print(f"User asked: {prompt}")

    # Step 1: Run moderation check
    mod_result = client.moderations.create(
        model="omni-moderation-latest",
        input=prompt
    )

    print(f"Moderation result: {mod_result}")
    print(f"mod_result.results[0].flagged result: {mod_result.results[0].flagged}")
    # Check if flagged
    if mod_result.results[0].flagged:
        print("Prompt was flagged by moderation. Blocking response.")
        return "Sorry, your request cannot be processed as it violates safety policies."
    else:
        print("Prompt passed moderation.")
        # Step 2: Only query GPT if safe
        response = client.chat.completions.create(
            model="gpt-5-nano",  # Use your desired model
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
                {"role": "user", "content": prompt}
            ]
        )

        answer = response.choices[0].message.content
        print(f"Assistant replied: {answer}")
        return answer

In [6]:
#print(ask_question("You're really annoying and I don’t want to work with you anymore."))  # Will be flagged by moderation
print(ask_question("I am so angry that I want to kill you"))  # Will not pass moderation

User asked: I am so angry that I want to kill you
Moderation result: ModerationCreateResponse(id='modr-435', model='omni-moderation-latest', results=[Moderation(categories=Categories(harassment=True, harassment_threatening=True, hate=False, hate_threatening=False, illicit=False, illicit_violent=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, harassment/threatening=True, hate/threatening=False, illicit/violent=False, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'